## Read Json File

In [57]:
import pandas as pd
import json

In [58]:
df1 = pd.read_json('/Users/cindy/Desktop/Fetch/receipts.json', lines = True)
df2 = pd.read_json('/Users/cindy/Desktop/Fetch/brands.json', lines = True)
df3 = pd.read_json('/Users/cindy/Desktop/Fetch/users.json', lines = True)

## Preprocessing Receipts Table

In [59]:

## Extract id from json nesting

df1['_id'] = df1['_id'].apply(lambda x: x['$oid']if isinstance(x, dict)\
                                              and '$oid' in x else x)





## Extract date from json nesting and convert unix timestamp to datetime for all date column

timestamp_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']

for column in timestamp_columns:
    df1[column] = df1[column].apply(lambda x: x['$date']if isinstance(x, dict)\
                                              and '$date' in x else x)
    df1[column] = pd.to_datetime(df1[column], unit='ms').dt.date
    
    
## drop duplicated receipt ID and 
## Will extract data from 'rewardsReceiptItemList' later, drop it to reduce date redandency 

receipts = df1.rename(columns = {'_id':'receipt_id'})
receipts.drop('rewardsReceiptItemList', axis = 1, inplace = True)
receipts.drop_duplicates(subset = ['receipt_id'], inplace = True)
receipts.head(5)

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,500.0,2021-01-03,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,150.0,2021-01-02,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,NaT,2021-01-03,NaT,5.0,2021-01-03,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,5.0,2021-01-03,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,5.0,2021-01-02,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


## Preprocessing Brands data and change the name to products

In [60]:

## Extract id from json nesting

df2['_id'] = df2['_id'].apply(lambda x : x['$oid'] if isinstance(x,dict)\
                              and '$oid' in x else x)
df2['cpg'] = df2['cpg'].apply(lambda x : x['$id'] if isinstance(x,dict)\
                              and '$id' in x else x)
df2['cpg'] = df2['cpg'].apply(lambda x : x['$oid'] if isinstance(x,dict)\
                              and '$oid' in x else x)

products = df2.rename(columns = {'_id':'brand_id'})
products.head(5)

,brand_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


## Preprocessing Users Table

In [61]:
df3['_id'] = df3['_id'].apply(lambda x : x['$oid'] if isinstance(x,dict)\
                              and '$oid' in x else x)
df3['createdDate'] = df3['createdDate'].apply(lambda x: x['$date'] if isinstance(x, dict)\
                                              and '$date' in x else x)
df3['lastLogin'] = df3['lastLogin'].apply(lambda x: x['$date'] if isinstance(x, dict)\
                                              and '$date' in x else x)

users = df3.rename(columns = {'_id':'user_id'})
users['createdDate'] = pd.to_datetime(users['createdDate'], unit='ms').dt.date
users['lastLogin'] = pd.to_datetime(users['lastLogin'], unit='ms').dt.date
users.drop_duplicates(subset = ['user_id'],inplace = True)
users.head(5)

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03,2021-01-03,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03,2021-01-03,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,2021-01-03,2021-01-03,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,2021-01-03,2021-01-03,consumer,Email,WI


## Getting Receipt_items Table

In [62]:
## Traverse row for 'rewardsReceiptItemList' column extract every item in each row, append to df4
## create receipt_id column that used to link with receipts table



df4_ = []

for index, row in df1.iterrows():
    if 'rewardsReceiptItemList' in row and isinstance(row['rewardsReceiptItemList'],list)\
    and len(row['rewardsReceiptItemList']) > 0:
        
        for item in row['rewardsReceiptItemList']:
            item['receipt_id'] = row['_id']
            df4_.append(item)

            
            
## reset_index and rename index to 'item_id'


df4 = pd.DataFrame(df4_)
df4.reset_index(inplace = True)
df4.rename(columns = {'index':'item_id'}, inplace = True)


## select useful column
receipt_items = df4[['receipt_id','item_id','barcode','userFlaggedBarcode','finalPrice','itemPrice','quantityPurchased']]
receipt_items.head(5)

,receipt_id,item_id,barcode,userFlaggedBarcode,finalPrice,itemPrice,quantityPurchased
0,5ff1e1eb0a720f0523000575,0,4011,4011,26.00,26.00,5.0
1,5ff1e1bb0a720f052300056b,1,4011,NaN,1,1,1.0
2,5ff1e1bb0a720f052300056b,2,028400642255,028400642255,10.00,10.00,1.0
3,5ff1e1f10a720f052300057a,3,NaN,4011,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4,4011,4011,28.00,28.00,4.0


## Output CSV Format for Postgre

In [63]:
receipts.to_csv('/Users/cindy/Desktop/Fetch/receipts.csv', index = False)
users.to_csv('/Users/cindy/Desktop/Fetch/users.csv',index = False)
products.to_csv('/Users/cindy/Desktop/Fetch/products.csv',index = False)
receipt_items.to_csv('/Users/cindy/Desktop/Fetch/receipt_items.csv',index = False)